In [7]:
# Use "jupyter_plotly_dash" for Apporto.
# Use "jupyter_dash" for local machine.
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


from Animal_Shelter import AnimalShelter

# initialize shelter for database queries
username = "aacuser"
password = "userPass"
shelter = AnimalShelter(username, password)

####################
# Helper Functions #
####################
def filterOutput(filterOpt):
    if filterOpt == "waterDog":
        outputFrame = pd.DataFrame.from_records(shelter.readAll(
            {'animal_type': 'Dog',
             'breed': {'$in': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
             'sex_upon_outcome': 'Intact Female',
             'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
             }))
    elif filterOpt == "mtnWldDog":
        outputFrame = pd.DataFrame.from_records(shelter.readAll(
            {'animal_type': 'Dog',
             'breed': {'$in': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
             'sex_upon_outcome': 'Intact Male',
             'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
             }))
    elif filterOpt == "disTrkgDog":
        outputFrame = pd.DataFrame.from_records(shelter.readAll(
            {'animal_type': 'Dog',
             'breed': {'$in': ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
             'sex_upon_outcome': 'Intact Male',
             'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
             }))
    else:
        outputFrame = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}, True))


    outputFrame = outputFrame[["animal_id", "breed", "name", "sex_upon_outcome", "age_upon_outcome_in_weeks", "location_lat", "location_long"]]
    outputFrame.age_upon_outcome_in_weeks = outputFrame.age_upon_outcome_in_weeks.round()
    #df.location_lat = df.location_lat.round(2)
    #df.location_long = df.location_long.round(2)
    outputFrame.rename(columns = {'animal_id':'Animal ID', 'breed':'Breed', 'name':'Name', 'sex_upon_outcome':'Outcome Sex',
                                  'age_upon_outcome_in_weeks':'Outcome Age', 'location_lat':'Latitude', 'location_long':'Longitude'}, inplace = True)
    return outputFrame



#############################
# Data Manipulation / Model #
#############################
df = filterOutput(None)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# logo for Grazioso Salvare company
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div([                      # main div for entire header
        html.Div(                   # div for logo
            html.A(                 # makes logo a hyperlink
                html.Img(id='customer-image',
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    alt='customer image',
                    style={'display':'flex', 'max-width': '100%'}),
                href='https://www.snhu.edu/', target="_blank",
                style={'display':'flex', 'max-width': '200px', 'max-height': '100%'}),
            style={'display':'flex', 'max-width': '200px', 'max-height': '100%'}),

        # div for the title
        html.Div(html.B(html.H1('Service Dog Finder'))),

        # div for author
        html.Div(html.B(html.H5('Dashboard Created By: Preston Burkhardt')),
                 style={'display':'flex', 'align-self': 'flex-end'})
    ],
    # style information for the formatting of the header div
    style={'display':'flex', 'flex-direction': 'row', 'justify-content': 'space-between', 'justify-items':'center', 'align-items':'center', 'width': '100%', 'height': '10rem'}),

    html.Hr(),

    # radio item/filter for displaying certain categories of animals
    html.Div(
        dcc.RadioItems(
            id='SnRtype',
            # sets the options and the values they correspond to
            options=[
                {'label': 'All', 'value': 'allDogs'},
                {'label': 'Water Rescue', 'value': 'waterDog'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mtnWldDog'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disTrkgDog'}
            ],
            value='allDogs',
            inline=True,
            style={'display': 'flex', 'justify-content': 'space-evenly'})),

    html.Hr(),

    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'left'}
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here

    ),
    html.Br(),
    html.Hr(),

    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
])

###############################################
# Interaction Between Components / Controller #
###############################################

# Changes the table based on filters in filterOutput() function.
# Based on the values in the radio item/menu.
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('SnRtype', 'value')])
def update_dashboard(filter_type):
    df = filterOutput(filter_type)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return (data,columns)

# Callback for highlighting table cells that have been clicked on.
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    pass
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(
    #        figure = ###
    #    )
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    pass
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.



if __name__ == '__main__':
    # Use "app" for Apporto.
    # Use "app.run_server()" for local machine.
    #app
    app.run_server(host="localhost",port=8051)

Dash app running on http://localhost:8051/
